In [1]:
# https://www.datacamp.com/tutorial/random-forests-classifier-python

import pandas as pd
import numpy as np

# data_handle = 'cond1_pat59to74'
data_handle = 'cond1_pat1to81'
pca_mode = 'pca'

inter_dir = '../../data/intermediates'
data_subdir = data_handle + '_' + pca_mode
data_dir = inter_dir + '/' + data_subdir

fX_test = data_dir + '/X_test_' + data_handle + '.csv'
fX_train = data_dir + '/X_train_' + data_handle + '.csv'
fy_test = data_dir + '/y_test_' + data_handle + '.csv'
fy_train = data_dir + '/y_train_' + data_handle + '.csv'

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
X_test = np.loadtxt(fX_test, delimiter=',')
X_train = np.loadtxt(fX_train, delimiter=',')
y_test = pd.read_csv(fy_test)
y_train = pd.read_csv(fy_train)

print('---- X_test ----')
print(X_test, '\n')
print('---- X_train ----')
print(X_train, '\n')
print('---- y_test ----')
display(y_test)
print('---- y_train ----')
display(y_train)

---- X_test ----
[[ 1.18913033e+01 -3.13923072e+01  2.86936299e+01 ...  1.06612242e+01
  -1.05600398e+01 -3.49576430e-01]
 [-5.76585568e+01  1.05648038e+01 -5.64950233e+01 ... -2.50804358e+00
  -1.70640312e-01 -1.19032010e-01]
 [ 6.43475079e+01  8.03819159e+00  1.76904290e+02 ...  1.72141093e+00
  -7.74062066e+00  1.03485685e+00]
 ...
 [-3.09248012e+01 -4.92120982e+00  6.73864288e+00 ...  1.45920023e+00
   9.37529032e-01  1.70313407e-01]
 [ 2.97478005e+02  2.32131163e+02 -7.43191036e+01 ... -9.45949761e+00
   3.49333979e+00  4.51864437e+00]
 [ 3.31884623e+01 -3.86639501e+01  8.64254408e+01 ...  6.21053054e+00
   2.26404154e+00 -1.21584567e-01]] 

---- X_train ----
[[-3.37120081e+01 -5.83834682e+00 -6.13892836e+01 ...  1.95462300e+01
  -2.13193920e+01  4.40214035e-14]
 [-2.12993489e+00 -2.90888782e+01  4.57751694e+00 ... -3.96749102e+00
   1.65920400e+00  4.40214035e-14]
 [-5.74174778e+01  3.15938291e+01 -3.56669380e+01 ... -2.98639827e+00
  -1.45455239e+01  4.40214035e-14]
 ...
 [-7.68

,subject,class
0,79,1
1,18,0
2,10,0
3,6,0
4,12,0
5,61,0
6,9,0
7,29,1
8,3,0
9,14,0


---- y_train ----


,subject,class
0,44,1
1,63,0
2,76,1
3,59,0
4,31,1
...,...,...
59,45,1
60,16,0
61,34,1
62,23,0


In [4]:
%%time

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train['class'])

CPU times: user 896 ms, sys: 147 ms, total: 1.04 s
Wall time: 888 ms


RandomForestClassifier()

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test['class'], y_pred)
print("Accuracy:", accuracy)

In [ ]:
%%time

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {'n_estimators': randint(1,500), 'max_depth': randint(1,20)}

rf = RandomForestClassifier()
rand_search = RandomizedSearchCV(rf, param_distributions = param_dist, n_iter=5, cv=5)

rand_search.fit(X_train, y_train['class'])

In [ ]:
best_est = rand_search.best_estimator_
best_params = rand_search.best_params_
print('Best hyperparameters:', best_params)

In [ ]:
%%time
rf_best = RandomForestClassifier(max_depth=best_params['max_depth'], n_estimators=best_params['n_estimators'])
# rf_best = RandomForestClassifier(max_depth=100, n_estimators=4)  # inconsistent results upon repeat
rf_best.fit(X_train, y_train['class'])
y_pred_best = rf_best.predict(X_test)
accuracy_best = accuracy_score(y_test['class'], y_pred_best)
print("Best accuracy:", accuracy_best)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test['class'], y_pred_best)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

In [ ]:
# F- I-- N---